<a href="https://colab.research.google.com/github/JakubPac/kurs_ml/blob/main/PCA_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
import plotly.express as px

In [4]:
from sklearn.datasets import load_iris

In [5]:
raw_data = load_iris()
all_data = raw_data.copy()

In [7]:
all_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [8]:
import plotly.graph_objects as go

In [12]:
data = all_data['data']
target = all_data['target']
target_names = all_data['target_names']
feature_names = [name.replace(' ', '_')[:-5] for name in all_data['feature_names']]
df = pd.DataFrame(np.c_[data, target], columns = feature_names + ['class'])
df['class'] = df['class'].map({0.0 : 'setosa', 1.0 : 'versicolor', 2.0 : 'virginica'})
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [16]:
px.scatter_3d(df, x = 'sepal_length', y = 'petal_length', z = 'petal_width', template = 'plotly_dark',
             title = 'Iris data - wizualizacja 3D', color = 'class', symbol = 'class', opacity = 0.5, width = 900, height = 500)

Standaryzacja

In [17]:
from sklearn.preprocessing import StandardScaler

X = df.iloc[:, [0,2,3]]
y = df.iloc[:, -1]

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std[:5]

array([[-0.90068117, -1.34022653, -1.3154443 ],
       [-1.14301691, -1.34022653, -1.3154443 ],
       [-1.38535265, -1.39706395, -1.3154443 ],
       [-1.50652052, -1.2833891 , -1.3154443 ],
       [-1.02184904, -1.34022653, -1.3154443 ]])

Implementacja PCA

In [19]:
#macierz kowariancji
cov_mat = np.cov(X_std, rowvar = False)
cov_mat

array([[1.00671141, 0.87760447, 0.82343066],
       [0.87760447, 1.00671141, 0.96932762],
       [0.82343066, 0.96932762, 1.00671141]])

In [24]:
# wektory i wartosci wlasne
eig_vals, eig_vects = np.linalg.eig(cov_mat)

print(f'wartosci wlasne: {eig_vals}\n')

print(f'wektory wlasne: \n{eig_vects}')

wartosci wlasne: [2.78833033 0.20075012 0.03105378]

wektory wlasne: 
[[ 0.55964149  0.81270446  0.16221241]
 [ 0.59148855 -0.2546058  -0.76506024]
 [ 0.58046765 -0.52410624  0.62319335]]


In [25]:
# posortowanie wektorów wedlug wartosci
eig_pairs = [(np.abs(eig_vals[i]), eig_vects[i]) for i in range(len(eig_vals))]
eig_pairs.sort(reverse = True)
eig_pairs

[(np.float64(2.7883303296752913), array([0.55964149, 0.81270446, 0.16221241])),
 (np.float64(0.20075011806343807),
  array([ 0.59148855, -0.2546058 , -0.76506024])),
 (np.float64(0.031053780449190244),
  array([ 0.58046765, -0.52410624,  0.62319335]))]

In [26]:
# wartosc procentowa wyjasnionej wariancji

total = sum(eig_vals)
explained_variance_ratio = [(i/total) for i in sorted(eig_vals, reverse = True)]
explained_variance_ratio

[np.float64(0.9232471536035964),
 np.float64(0.06647059464767172),
 np.float64(0.010282251748731881)]

In [27]:
cumulative_explained_variance = np.cumsum(explained_variance_ratio)
cumulative_explained_variance

array([0.92324715, 0.98971775, 1.        ])

In [28]:
results = pd.DataFrame(data = {'explained_variance_ratio': explained_variance_ratio})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.923247,0.923247,1
1,0.066471,0.989718,2
2,0.010282,1.000000,3


In [31]:
fig = go.Figure(data = [go.Bar(x = results['component'], y = results['explained_variance_ratio'], name = 'explained varaince ratio'),
              go.Scatter(x = results['component'], y = results['cumulative'], name = 'cumulative explained variance')],
              layout = go.Layout(title = 'PCA - 3 components', width = 900, height = 500, template = 'plotly_dark'))
fig.show()

In [32]:
eig_pairs

[(np.float64(2.7883303296752913), array([0.55964149, 0.81270446, 0.16221241])),
 (np.float64(0.20075011806343807),
  array([ 0.59148855, -0.2546058 , -0.76506024])),
 (np.float64(0.031053780449190244),
  array([ 0.58046765, -0.52410624,  0.62319335]))]

In [33]:
# 2 komponenty, W - macierz skladajaca sie z 2 wektorów wlasnych majacych najwieksza wartosc wlasna

W = np.hstack((eig_pairs[0][1].reshape(3,1), eig_pairs[1][1].reshape(3,1)))
W

array([[ 0.55964149,  0.59148855],
       [ 0.81270446, -0.2546058 ],
       [ 0.16221241, -0.76506024]])

In [34]:
X_pca = X_std.dot(W)
pca_df = pd. DataFrame(data = X_pca, columns = ['pca_1', 'pca_2'])
pca_df['class'] = df['class']
pca_df['pca_2'] = - pca_df['pca_2']
pca_df

,pca_1,pca_2,class
0,-1.806648,-0.814881,setosa
1,-1.942269,-0.671542,setosa
2,-2.124082,-0.542674,setosa
3,-2.099509,-0.442063,setosa
4,-1.874459,-0.743212,setosa
...,...,...,...
145,1.482018,0.703149,virginica
146,1.032982,0.558058,virginica
147,1.282340,0.544368,virginica
148,1.235349,1.090439,virginica


In [36]:
px.scatter(pca_df, 'pca_1', 'pca_2', color = 'class', template = 'plotly_dark', width = 900)

PCA - scikit-learn

In [37]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X_pca = pca.fit_transform(X_std)
pca_dfg = pd.DataFrame(data = X_pca, columns = ['pca_1', 'pca_2'])
pca_df['class'] = df['class']
pca_df

,pca_1,pca_2,class
0,-1.806648,-0.814881,setosa
1,-1.942269,-0.671542,setosa
2,-2.124082,-0.542674,setosa
3,-2.099509,-0.442063,setosa
4,-1.874459,-0.743212,setosa
...,...,...,...
145,1.482018,0.703149,virginica
146,1.032982,0.558058,virginica
147,1.282340,0.544368,virginica
148,1.235349,1.090439,virginica


In [38]:
px.scatter(pca_df, 'pca_1', 'pca_2', color = 'class', width = 900, template = 'plotly_dark')